In [10]:
import os
import time
import subprocess
import pandas as pd
pep_folder = 'data/1k-species-data/test'

for pep_file in os.listdir(pep_folder):
    if not pep_file.endswith('.final.pep'):
        continue
    
    if pep_file.startswith('yH'):
        species_name = pep_file.split('_', 1)[1].rsplit('_', 1)[0]
        for other_file in os.listdir(pep_folder):
            if species_name in other_file and 'final' in other_file and not other_file.startswith('yH'):
                pep_file = other_file
    else:
        species_name = pep_file.rsplit('.', 2)[0]
                
    print(pep_file)
    print(species_name)

brettanomyces_anomalus.final.pep
brettanomyces_anomalus
arxula_adeninivorans.final.pep
arxula_adeninivorans
alloascoidea_hylecoeti.final.pep
alloascoidea_hylecoeti
ashbya_aceri.final.pep
ashbya_aceri
candida_albicans.final.pep
candida_albicans


In [ ]:
import os
import time
import subprocess
import pandas as pd

pep_folder = 'data/1k-species-data/y1000p_pep_files'
db_folder = 'data/1k-species-data/blast_dbs'
hits_folder = 'data/1k-species-data/CHC1_hits'
master_fasta = 'data/1k-species-data/1k-CHC1.fasta'
metadata_csv = 'data/1k-species-data/CHC1_metadata.csv'
query_fasta = 'data/1k-species-data/CHC1_seed.fasta'

'''
Add fuzzy-string logic (fuzzy wuzzy, other tool) to account for human error.
'''

metadata = []
species_found = []

for pep_file in os.listdir(pep_folder):
    if not pep_file.endswith('.final.pep'):
        continue
    
    if pep_file.startswith('yH'):
        species_name = pep_file.split('_', 1)[1].rsplit('_', 1)[0]
        for other_file in os.listdir(pep_folder):
            if species_name in other_file and 'final' in other_file and not other_file.startswith('yH'):
                pep_file = other_file
    else:
        species_name = pep_file.rsplit('.', 2)[0]
                
    print(pep_file)
    print(species_name)

    if species_name in species_found:
        continue
    species_found.append(species_name)
    print(f'Processing {species_name}')
    pep_path = os.path.join(pep_folder, pep_file)

    species_folder = os.path.join(db_folder, species_name)
    os.makedirs(species_folder, exist_ok=True)
    db_path = os.path.join(species_folder, species_name)
    subprocess.run([
        'makeblastdb',
        '-in', pep_path,
        '-dbtype', 'prot',
        '-out', db_path
    ], check=True)
    
    hits_file = os.path.join(hits_folder, f"{species_name}_CHC1_hits.txt")
    subprocess.run([
        'blastp',
        '-query', query_fasta,
        '-db', db_path,
        '-out', hits_file,
        '-outfmt', '6'
    ], check=True)

    time.sleep(0.1)
    
    if os.path.exists(hits_file):
        with open(hits_file) as f:
            lines = f.readlines()
        if lines:
            # BLAST outfmt 6: qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore
            top_hit = max(lines, key=lambda x: float(x.split()[2]))  # highest % identity
            cols = top_hit.strip().split()
            sseqid, pident, evalue, bitscore = cols[1], float(cols[2]), float(cols[10]), float(cols[11])
            
            seq_lines = []
            record = False
            with open(pep_path) as pep_f:
                for line in pep_f:
                    if line.startswith('>'):
                        record = sseqid in line
                        continue
                    if record:
                        seq_lines.append(line.strip())
            seq = '\n'.join([line.strip().rstrip('*') for line in seq_lines])
            
            with open(master_fasta, 'a') as f_out:
                f_out.write(f'>{species_name}|{sseqid}\n{seq}\n')
            
            metadata.append([species_name, sseqid, len(seq.replace('\n','')), pident, evalue, bitscore])
        else:
            print(f'No hits found for {species_name}')

df = pd.DataFrame(metadata, columns=['Species', 'Hit_ID', 'Seq_Length', 'Percent_Identity', 'Evalue', 'Bitscore'])
df.to_csv(metadata_csv, index=False)

In [33]:
from Bio import AlignIO
from Bio.Align import MultipleSeqAlignment
from Bio.Align import substitution_matrices
from Bio.Align.AlignInfo import SummaryInfo

alignment = AlignIO.read('data/Sporopachydermia-clade_CHC_manual_consensus.fasta', 'fasta')

print(f'Number of sequences: {len(alignment)}')
print(f'Alignment length: {alignment.get_alignment_length()}')

seq_weight = 1 # Enable different weights for different sequences
blosum62 = substitution_matrices.load('BLOSUM62')
plurality_value = 0.5 * (len(alignment) * seq_weight)
consensus_seq = ''

for i in range(alignment.get_alignment_length()):
    site = alignment[:, i]
    if all(aa == '-' for aa in site):
        consensus_seq = consensus_seq + '-'
        continue
    elif site.count('-') == 1:
        for aa in site: # Fix such that this code works for more than 2 sequences
            if aa != '-':
                consensus_seq = consensus_seq + aa
        continue
    scores = {}
    for k, residue in enumerate(site):
        scores[residue] = 0
        positive_matches = 0
        for j, other_residue in enumerate(site):
            if k == j:
                continue
            scores[residue] += blosum62[residue, other_residue] * seq_weight
            positive_matches += scores[residue] if scores[residue] > 0 else 0 # Fix

    if positive_matches > plurality_value:
        consensus_seq = consensus_seq + residue
    else:
        consensus_seq = consensus_seq + 'x'

Number of sequences: 2
Alignment length: 4194


In [34]:
print(consensus_seq)

>---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [35]:
print(blosum62)

#  Matrix made by matblas from blosum62.iij
#  * column uses minimum score
#  BLOSUM Clustered Scoring Matrix in 1/2 Bit Units
#  Blocks Database = /data/blocks_5.0/blocks.dat
#  Cluster Percentage: >= 62
#  Entropy =   0.6979, Expected =  -0.5209
     A    R    N    D    C    Q    E    G    H    I    L    K    M    F    P    S    T    W    Y    V    B    Z    X    *
A  4.0 -1.0 -2.0 -2.0  0.0 -1.0 -1.0  0.0 -2.0 -1.0 -1.0 -1.0 -1.0 -2.0 -1.0  1.0  0.0 -3.0 -2.0  0.0 -2.0 -1.0  0.0 -4.0
R -1.0  5.0  0.0 -2.0 -3.0  1.0  0.0 -2.0  0.0 -3.0 -2.0  2.0 -1.0 -3.0 -2.0 -1.0 -1.0 -3.0 -2.0 -3.0 -1.0  0.0 -1.0 -4.0
N -2.0  0.0  6.0  1.0 -3.0  0.0  0.0  0.0  1.0 -3.0 -3.0  0.0 -2.0 -3.0 -2.0  1.0  0.0 -4.0 -2.0 -3.0  3.0  0.0 -1.0 -4.0
D -2.0 -2.0  1.0  6.0 -3.0  0.0  2.0 -1.0 -1.0 -3.0 -4.0 -1.0 -3.0 -3.0 -1.0  0.0 -1.0 -4.0 -3.0 -3.0  4.0  1.0 -1.0 -4.0
C  0.0 -3.0 -3.0 -3.0  9.0 -3.0 -4.0 -3.0 -3.0 -1.0 -1.0 -3.0 -1.0 -2.0 -3.0 -1.0 -1.0 -2.0 -2.0 -1.0 -3.0 -3.0 -2.0 -4.0
Q -1.0  1.0  0.0  0.